<a href="https://colab.research.google.com/github/kusanorootbeer/tf2_nb_training/blob/master/tf2train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google drive mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# tensorflow v2.1 install

In [0]:
!pip install tensorflow==2.1.0

# get dataset

In [0]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00341/HAPT%20Data%20Set.zip .
!mkdir drive/My\ Drive/Dataset
!unzip 'HAPT Data Set.zip' -d drive/My\ Drive/Dataset
!rm -rf  HAPT\ Data\ Set.zip

--2020-01-24 11:40:13--  http://archive.ics.uci.edu/ml/machine-learning-databases/00341/HAPT%20Data%20Set.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79596192 (76M) [application/x-httpd-php]
Saving to: ‘HAPT Data Set.zip’

HAPT Data Set.zip   100%[===================>]  75.91M  10.6MB/s    in 8.1s    

2020-01-24 11:40:21 (9.32 MB/s) - ‘HAPT Data Set.zip’ saved [79596192/79596192]

--2020-01-24 11:40:21--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2020-01-24 11:40:21--
Total wall clock time: 8.3s
Downloaded: 1 files, 76M in 8.1s (9.32 MB/s)
Archive:  HAPT Data Set.zip
  inflating: drive/My Drive/Dataset/features_info.txt  
  inflating: drive/My Drive/Dataset/README.txt  
  inflating: drive/My Drive/Dataset/RawData/acc_exp01_user01.t

# model building

## import

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.1.0-rc1'

In [0]:
import tensorflow.keras as k
import numpy as np
import pandas as pd
import os 

## Data load

In [0]:
path = "/content/drive/My Drive/Dataset"
features_name = pd.read_csv("{}/features.txt".format(path), header=None)
activity_name = pd.read_csv("{}/activity_labels.txt".format(path), header=None)
train_x = np.loadtxt("{}/Train/X_train.txt".format(path))
train_y = np.loadtxt("{}/Train/y_train.txt".format(path)) -1
train_subject = np.loadtxt("{}/Train/subject_id_train.txt".format(path))
test_x = np.loadtxt("{}/Test/X_test.txt".format(path))
test_y = np.loadtxt("{}/Test/y_test.txt".format(path)) -1
test_subject = np.loadtxt("{}/Test/subject_id_test.txt".format(path))

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

## create model

In [0]:
class MLP_Classifier(tf.keras.Model):
  def __init__(self):
    super(MLP_Classifier, self).__init__()
    self.fc1 = k.layers.Dense(300, activation="relu")
    self.fc2 = k.layers.Dense(100, activation="relu")
    self.fc3 = k.layers.Dense(50, activation="relu")
    self.out_put = k.layers.Dense(12, activation="softmax")

  def call(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    return self.out_put(x)

model = MLP_Classifier()

In [0]:
loss_object = k.losses.SparseCategoricalCrossentropy()
optimizer = k.optimizers.Adam()

In [0]:
train_loss = k.metrics.Mean(name='train_loss')
train_accuracy = k.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = k.metrics.Mean(name='test_loss')
test_accuracy = k.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [0]:
@tf.function
def train_step(features, labels):
  with tf.GradientTape() as tape:
    predictions = model(features)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
  return tf.argmax(predictions, axis=1)

In [0]:
@tf.function
def test_step(features, labels):
  predictions = model(features)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)
  return tf.argmax(predictions, axis=1)

## training model

In [65]:
EPOCHS = 100

for epoch in range(EPOCHS):
  train_label_stack = []
  train_pred_stack = []
  test_label_stack = []
  test_pred_stack = []
  for features, labels in train_ds:
    pred =  train_step(features, labels)
    train_label_stack.append(labels)
    train_pred_stack.append(pred)
  train_label_stack = tf.concat(train_label_stack, axis=0)
  train_pred_stack = tf.concat(train_pred_stack,axis=0)

  for test_features, test_labels in test_ds:
    pred = test_step(test_features, test_labels)
    test_label_stack.append(test_labels)
    test_pred_stack.append(pred)
  test_label_stack = tf.concat(test_label_stack, axis=0)
  test_pred_stack = tf.concat(test_pred_stack ,axis=0)

  template = 'Epoch {}, Loss: {:.7}, Accuracy: {:.7}, Test Loss: {:.7}, Test Accuracy: {:.7}'
  print (template.format(epoch+1, train_loss.result(), train_accuracy.result()*100, test_loss.result(), test_accuracy.result()*100))
  
  train_mat = tf.math.confusion_matrix(train_label_stack, train_pred_stack,num_classes=12)
  test_mat = tf.math.confusion_matrix(test_label_stack, test_pred_stack, num_classes=12)

  # reset metrics
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.2914465, Accuracy: 89.12064, Test Loss: 0.2288379, Test Accuracy: 91.02888
Epoch 2, Loss: 0.1226014, Accuracy: 95.14613, Test Loss: 0.1848877, Test Accuracy: 93.07401
Epoch 3, Loss: 0.1267475, Accuracy: 94.9015, Test Loss: 0.3661979, Test Accuracy: 86.17963
Epoch 4, Loss: 0.1039158, Accuracy: 96.07314, Test Loss: 0.1962868, Test Accuracy: 92.56799
Epoch 5, Loss: 0.09944429, Accuracy: 95.97013, Test Loss: 0.2161215, Test Accuracy: 91.93549
Epoch 6, Loss: 0.09077867, Accuracy: 96.67825, Test Loss: 0.1920588, Test Accuracy: 93.45351
Epoch 7, Loss: 0.08780652, Accuracy: 96.56238, Test Loss: 0.1890313, Test Accuracy: 93.29538
Epoch 8, Loss: 0.09501967, Accuracy: 96.33063, Test Loss: 0.2379377, Test Accuracy: 91.27135
Epoch 9, Loss: 0.06810943, Accuracy: 97.33488, Test Loss: 0.2184059, Test Accuracy: 91.96711
Epoch 10, Loss: 0.07988268, Accuracy: 96.74263, Test Loss: 0.2973439, Test Accuracy: 91.0816
Epoch 11, Loss: 0.07501288, Accuracy: 96.80701, Test Loss: 0.2634796, Test 

## evaluate predistions

In [0]:
train_df = pd.DataFrame(data=train_mat.numpy(), columns=activity_name, index=activity_name)
test_df = pd.DataFrame(data=test_mat.numpy(), columns=activity_name, index=activity_name)

In [71]:
from IPython.display import display, HTML

print("train confusion matrix:")
display(train_df)
print("test confusion matrix:")
display(HTML(test_df.to_html()))

train confusion matrix:


,"(1 WALKING ,)","(2 WALKING_UPSTAIRS ,)","(3 WALKING_DOWNSTAIRS,)","(4 SITTING ,)","(5 STANDING ,)","(6 LAYING ,)","(7 STAND_TO_SIT ,)","(8 SIT_TO_STAND ,)","(9 SIT_TO_LIE ,)","(10 LIE_TO_SIT ,)","(11 STAND_TO_LIE ,)","(12 LIE_TO_STAND ,)"
"(1 WALKING ,)",1226,0,0,0,0,0,0,0,0,0,0,0
"(2 WALKING_UPSTAIRS ,)",0,1073,0,0,0,0,0,0,0,0,0,0
"(3 WALKING_DOWNSTAIRS,)",0,0,987,0,0,0,0,0,0,0,0,0
"(4 SITTING ,)",0,0,0,1276,17,0,0,0,0,0,0,0
"(5 STANDING ,)",0,0,0,17,1406,0,0,0,0,0,0,0
"(6 LAYING ,)",0,0,0,0,0,1413,0,0,0,0,0,0
"(7 STAND_TO_SIT ,)",0,0,0,0,0,0,47,0,0,0,0,0
"(8 SIT_TO_STAND ,)",0,0,0,0,0,0,0,23,0,0,0,0
"(9 SIT_TO_LIE ,)",0,0,0,0,0,0,0,0,75,0,0,0
"(10 LIE_TO_SIT ,)",0,0,0,0,0,0,0,0,0,60,0,0


test confusion matrix:


,"(1 WALKING ,)","(2 WALKING_UPSTAIRS ,)","(3 WALKING_DOWNSTAIRS,)","(4 SITTING ,)","(5 STANDING ,)","(6 LAYING ,)","(7 STAND_TO_SIT ,)","(8 SIT_TO_STAND ,)","(9 SIT_TO_LIE ,)","(10 LIE_TO_SIT ,)","(11 STAND_TO_LIE ,)","(12 LIE_TO_STAND ,)"
"(1 WALKING ,)",488,1,7,0,0,0,0,0,0,0,0,0
"(2 WALKING_UPSTAIRS ,)",25,444,1,0,0,0,0,0,0,0,1,0
"(3 WALKING_DOWNSTAIRS,)",5,16,399,0,0,0,0,0,0,0,0,0
"(4 SITTING ,)",0,1,0,458,47,0,0,2,0,0,0,0
"(5 STANDING ,)",0,0,0,31,524,0,1,0,0,0,0,0
"(6 LAYING ,)",0,1,0,0,4,540,0,0,0,0,0,0
"(7 STAND_TO_SIT ,)",0,1,0,2,1,0,17,1,0,0,1,0
"(8 SIT_TO_STAND ,)",0,1,0,0,0,0,1,8,0,0,0,0
"(9 SIT_TO_LIE ,)",0,1,0,0,0,0,0,0,24,0,6,1
"(10 LIE_TO_SIT ,)",0,0,0,0,0,0,0,0,1,17,0,7
